# GenAI v3: Consistent Video Background Manipulation

**Zero training required! No frame-by-frame inconsistency!**

## Key Features
- **CONSISTENT background** across ALL frames (no flicker!)
- **Video-native APIs**: Google Veo (Colab), Runway, or local SDXL
- **ONE background generated**, applied to entire scene
- **Product AUTO-DETECTED** using SAM + DINO
- **Outputs BOTH** video AND frame

## Backends (choose one)
| Backend | Best For | How |
|---------|----------|-----|
| `"google"` | Google Colab | Uses Vertex AI (Imagen/Veo) |
| `"runway"` | Runway users | Uses Runway Gen-3 API |
| `"consistent"` | Local GPU | Single SDXL generation |
| `"auto"` | Default | Tries Google, falls back |

## How It Works
- `increase`: Background → **SOLID GRAY** → all attention on product
- `decrease`: Background → **VIBRANT/COLORFUL** → attention diverts

In [ ]:
# For Google Colab: authenticate first
from google.colab import auth
auth.authenticate_user()

from GenAI_v3 import SceneManipulator

In [ ]:
# Option 1: Google Colab (recommended - uses Vertex AI)
manipulator = SceneManipulator(
    video_dir="data/data_tiktok",
    output_dir="outputs/genai_v3",
    backend="google",  # Uses Google Imagen/Veo
    # google_project_id="your-project-id",  # Auto-detected on Colab
)

# Option 2: Local SDXL (consistent single-generation)
# manipulator = SceneManipulator(
#     backend="consistent",  # Generate ONE bg, apply to all frames
#     device="cuda",
# )

# Option 3: Runway API
# manipulator = SceneManipulator(
#     backend="runway",
#     runway_api_key="your-api-key",
# )

## Increase Attention on Product

Makes background less distracting (muted, simple) → viewer focuses on product

In [ ]:
# INCREASE attention on product in scene 6
result = manipulator.manipulate(
    video_id="YOUR_VIDEO_ID",  # ← Change this
    scene_index=6,              # ← Change this
    action="increase",
)

# Both video and frame are output
print(f"Video: {result.video_path}")
print(f"Frame: {result.frame_path}")
print(f"Frames manipulated: {result.frames_manipulated}")

## Decrease Attention on Product

Makes background more interesting (vibrant, detailed) → viewer distracted from product

In [ ]:
# DECREASE attention on product in scene 3
result = manipulator.manipulate(
    video_id="YOUR_VIDEO_ID",  # ← Change this
    scene_index=3,              # ← Change this
    action="decrease",
)

# Both video and frame are output
print(f"Video: {result.video_path}")
print(f"Frame: {result.frame_path}")
print(f"Frames manipulated: {result.frames_manipulated}")

## Why This Is Better

**Old approach (frame-by-frame SDXL)**:
- Each frame generated separately → inconsistent backgrounds
- Slow: 30 frames = 30 SDXL runs
- Flickering and visual artifacts

**New approach (consistent background)**:
- Generate ONE background from reference frame
- Apply SAME background to ALL frames
- Fast: 1 SDXL run regardless of scene length
- No flicker, perfectly consistent!

In [ ]:
# Simple usage - just video_id, scene_index, action
result = manipulator.manipulate(
    video_id="YOUR_VIDEO_ID",
    scene_index=6,
    action="increase",
)

print(f"Video: {result.video_path}")
print(f"Frame: {result.frame_path}")

## Batch Processing

In [ ]:
import pandas as pd

# Load valid scenes
scenes_df = pd.read_csv("data/valid_scenes.csv")

# Process multiple videos
results = []
for video_id in scenes_df['video_id'].unique()[:3]:  # First 3 videos
    try:
        result = manipulator.manipulate(
            video_id=str(video_id),
            scene_index=1,  # First scene
            action="increase",
        )
        results.append(result)
        print(f"✓ {video_id}")
        print(f"  Video: {result.video_path}")
        print(f"  Frame: {result.frame_path}")
    except Exception as e:
        print(f"✗ {video_id}: {e}")

## Summary

### How It Works
1. Load video and detect scenes (PySceneDetect)
2. Auto-detect product using SAM + DINO
3. **Generate ONE reference background** from middle frame
4. **Apply SAME background** to ALL frames in scene
5. Composite: Product from original + Generated background
6. Export video and sample frame

### Key Advantage: CONSISTENCY
- No frame-by-frame generation = No flicker
- Product stays 100% unchanged (from original frames)
- Background is identical across all frames
- Perfect for A/B testing

### Backends
| Backend | Speed | Quality | Best For |
|---------|-------|---------|----------|
| Google | Fast | High | Colab users |
| Runway | Fast | High | API users |
| Consistent | Medium | Good | Local GPU |

### Performance
- **~30s per scene** (vs 5+ min with frame-by-frame)
- ~10GB GPU memory (consistent backend)
- No training required!